In [ ]:
import math
from graphviz import Digraph
import random


class Value:
    
    def __init__(self, data, _children=(), _op='',  name=''):
        self.data = data
        self.grad = 0.0
        self._prev = set(_children)  # parents
        self._op = _op               # operation
        self.name = name
        self._backward = lambda: None  # will define later

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    # Addition
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward
        return out

    # Subtraction
    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return self + (-other)

    # Negation
    def __neg__(self):
        out = Value(-self.data, (self,), 'neg')
        def _backward():
            self.grad += -1 * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return self * (other ** -1)


    def __pow__(self, n):
        assert isinstance(n, (int, float))
        out = Value(self.data ** n, (self,), f'**{n}')

        def _backward():
            self.grad += n * (self.data ** (n - 1)) * out.grad

        out._backward = _backward
        return out
    # Multiplication
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - out.data**2) * out.grad

        out._backward = _backward
        return out

    # Backpropagation
    def backward(self):
        topo = []
        visited = set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)
        build(self)

        # Seed gradient
        self.grad = 1.0

        # Traverse in reverse topological order
        for v in reversed(topo):
            v._backward()






          # Right-side operations for Python numbers
    def __radd__(self, other): 
        return self + other

    def __rsub__(self, other): 
        return Value(other) - self

    def __rmul__(self, other): 
        return self * other

    def __rtruediv__(self, other): 
        return Value(other) / self

    def __rpow__(self, other):
        return Value(other) ** self


def trace(root):
        nodes, edges = set(), set()

        def build(v):
            if v not in nodes:
                nodes.add(v)
                for child in v._prev:
                    edges.add((child, v))
                    build(child)

        build(root)
        return nodes, edges



def draw_dot(root):
    dot = Digraph(format="svg", graph_attr={"rankdir": "LR"})
    nodes, edges = trace(root)

    for n in nodes:
        uid = str(id(n))
        # Create the 3-field rectangle
        var_name = n.name if n.name else ''
        label = f"{{ {var_name} | data={n.data:.4f} | grad={n.grad:.4f} }}"
        dot.node(name=uid, label=label, shape='record')

        # If there's an operation, create a circle for it
        if n._op:
            op_uid = uid + "_op"
            dot.node(name=op_uid, label=n._op, shape='circle')
            # Connect operation circle to this value rectangle
            dot.edge(op_uid, uid)
            # Connect operands (children) to the operation circle
            for child in n._prev:
                dot.edge(str(id(child)), op_uid)

    return dot


  

# Inputs
a = Value(0.5001, name='a')
b = Value(0.71, name='b')

# Compute intermediates
t1 = a * b
t1.name = 't1'

t2 = t1 + b
t2.name = 't2'

c = t2 + a
c.name = 'c'

d = c.tanh()
d.name = 'd'

# Forward pass
print("Forward pass:")
print("a:", a.data)
print("b:", b.data)
print("c:", c.data)
print("d:", d.data)

# Backward pass
d.backward()
print("\nBackward pass (gradients):")
print("a.grad:", a.grad)
print("b.grad:", b.grad)
print("c.grad:", c.grad)
print("d.grad:", d.grad)

# Draw graph
draw_dot(d)

x1 = Value(0.5, name='x1')
x2 = Value(0.7, name='x2')

# Weights
w1 = Value(random.uniform(-1, 1), name='w1')
w2 = Value(random.uniform(-1, 1), name='w2')
b  = Value(random.uniform(-1, 1), name='b')  # bias

# Forward pass with descriptive names
x1w1 = x1 * w1
x1w1.name = 'x1*w1'

x2w2 = x2 * w2
x2w2.name = 'x2*w2'

x1w1_x2w2 = x1w1 + x2w2
x1w1_x2w2.name = 'x1*w1 + x2*w2'

x1w1_x2w2_b = x1w1_x2w2 + b
x1w1_x2w2_b.name = 'x1*w1 + x2*w2 + b'

out = x1w1_x2w2_b.tanh()
out.name = 'tanh(x1*w1 + x2*w2 + b)'

# Display forward pass
print("Forward pass:")
for v in [x1, x2, w1, w2, b, x1w1, x2w2, x1w1_x2w2, x1w1_x2w2_b, out]:
    print(f"{v.name}: {v.data:.4f}")

# Backward pass
out.backward()

# Display gradients
print("\nBackward pass (gradients):")
for v in [x1, x2, w1, w2, b, x1w1, x2w2, x1w1_x2w2, x1w1_x2w2_b, out]:
    print(f"{v.name}.grad: {v.grad:.4f}")

# Draw graph
draw_dot(out)
